<a href="https://colab.research.google.com/github/pravincoder/Nano-GPT_Mental_health/blob/main/Mental_health_gpt_testing_notebook.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Input txt files includes Mental Health Q&A Samples Count=3.51k
with open('/content/drive/MyDrive/input.txt') as f:
  data=f.read()


In [ ]:
print("Total number of Characters in Txt File:- ",len(data))

Total number of Characters in Txt File:-  4686471


In [ ]:
# Lets Look First 1000 char/letters
print(data[:1000])

Question :- I'm going through some things with my feelings and myself. I barely sleep and I do nothing but think about how I'm worthless and how I shouldn't be here.
   I've never tried or contemplated suicide. I've always wanted to fix my issues, but I never get around to it.
   How can I change my feeling of being worthless to everyone?

Answer :- If everyone thinks you're worthless, then maybe you need to find new people to hang out with.Seriously, the social context in which a person lives is a big influence in self-esteem.Otherwise, you can go round and round trying to understand why you're not worthless, then go back to the same crowd and be knocked down again.There are many inspirational messages you can find in social media.  Maybe read some of the ones which state that no person is worthless, and that everyone has a good purpose to their life.Also, since our culture is so saturated with the belief that if someone doesn't feel good about themselves that this is somehow terrible

In [ ]:
# get Unique Char and its Count in data
chars = sorted(list(set(data)))
vocab_size=len(chars)
print(f"Unique Char :- {chars}")
print(f"Vocab_size :- {vocab_size}")

Unique Char :- ['\n', ' ', '!', '"', '#', '$', '%', '&', "'", '(', ')', '*', '+', ',', '-', '.', '/', '0', '1', '2', '3', '4', '5', '6', '7', '8', '9', ':', ';', '<', '=', '>', '?', '@', 'A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'J', 'K', 'L', 'M', 'N', 'O', 'P', 'Q', 'R', 'S', 'T', 'U', 'V', 'W', 'X', 'Y', 'Z', '[', ']', '_', 'a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l', 'm', 'n', 'o', 'p', 'q', 'r', 's', 't', 'u', 'v', 'w', 'x', 'y', 'z', '{', '}', '~', '\xa0', '¡', '·', '¿', 'Ú', 'á', 'é', 'í', 'ñ', 'ó', 'ú', 'ü', '–', '—', '‘', '’', '“', '”', '…']
Vocab_size :- 111


In [ ]:
# Construct Tokenizer, an Encoder & Decoder (Based on Each character)
stoi={ch:i for i,ch in enumerate(chars)}
itos = {i:ch for i,ch in enumerate(chars)}
encoder = lambda s: [stoi[c] for c in s]
decoder = lambda s: ''.join([itos[i] for i in s])
print(encoder(" hi hi"))
print(decoder(encoder(" hi hi")))

[1, 70, 71, 1, 70, 71]
 hi hi


In [ ]:
# Encode the entire Data and Store it in torch.temsor
import torch
data = torch.tensor(encoder(data),dtype=torch.long)
print(data.shape,data.dtype)
data[:100]

torch.Size([4686471]) torch.int64


tensor([50, 83, 67, 81, 82, 71, 77, 76,  1, 27, 14,  1, 42,  8, 75,  1, 69, 77,
        71, 76, 69,  1, 82, 70, 80, 77, 83, 69, 70,  1, 81, 77, 75, 67,  1, 82,
        70, 71, 76, 69, 81,  1, 85, 71, 82, 70,  1, 75, 87,  1, 68, 67, 67, 74,
        71, 76, 69, 81,  1, 63, 76, 66,  1, 75, 87, 81, 67, 74, 68, 15,  1, 42,
         1, 64, 63, 80, 67, 74, 87,  1, 81, 74, 67, 67, 78,  1, 63, 76, 66,  1,
        42,  1, 66, 77,  1, 76, 77, 82, 70, 71])

In [ ]:
# Train test Split
n = int(0.9 * len(data))
train_data = data[:n]
test_data = data[n:]


In [ ]:
block_size = 9
train_data[:block_size+1]

tensor([50, 83, 67, 81, 82, 71, 77, 76,  1, 27])

In [ ]:
torch.manual_seed(42)
batch_size = 4
block_size = 8

def get_batch(split):
  data = train_data if split=='train' else test_data
  ix = torch.randint(len(data)-block_size,(batch_size,))
  x = torch.stack([data[i:i+block_size] for i in ix])
  y = torch.stack([data[i+1:i+block_size+1] for i in ix])
  return x,y
xb,yb = get_batch('train')
print('inputs:-')
print(xb)

print('Target:-')
print(yb)

inputs:-
tensor([[71, 76, 69,  1, 63, 76, 66,  1],
        [76,  1, 70, 67, 74, 78,  1, 87],
        [85, 70, 63, 82,  1, 87, 77, 83],
        [ 8, 80, 67,  1,  3, 67, 75, 64]])
Target:-
tensor([[ 76,  69,   1,  63,  76,  66,   1,  71],
        [  1,  70,  67,  74,  78,   1,  87,  77],
        [ 70,  63,  82,   1,  87,  77,  83, 107],
        [ 80,  67,   1,   3,  67,  75,  64,  63]])


In [ ]:
import torch.nn as nn
from torch.nn import functional as F

class BigramLanguageModel(nn.Module):
  def __init__(self,vocab_size):
    super().__init__()
    self.token_embedding_table = nn.Embedding(vocab_size,vocab_size) # n_embed = num of embedding dimension

  def forward(self,idx,targets=None):
    # idx and targets are both (B,T) tensor of integers
    logits = self.token_embedding_table(idx)#(B,T,C)
    if targets is None:
      loss=None
    else:
      B,T,C = logits.shape
      logits = logits.view(B*T,C) # Now (batch_size = 32 = B*T )
      targets = targets.view(B*T)
      loss = F.cross_entropy(logits,targets) # -log probablity
    return logits,loss
  def genrate(self,idx,max_new_tokens):
    # idx is (B,T) array of indices in the Current Context
    for _ in range(max_new_tokens):
      # get the predictions
      logits,loss=self(idx)
      #print(logits)
      #print(loss)
      # focus only on the last time step
      logits=logits[:,-1,:] # Now it (B,C)
      #print(logits)
      # apply softmax to probabilities
      probs = F.softmax(logits,dim=-1)
      # sample from the distribution
      idx_next = torch.multinomial(probs,num_samples=1) # (B,1)
      idx=torch.cat((idx,idx_next),dim=1) #(B,T+1)
    return idx


m = BigramLanguageModel(vocab_size)
logits,loss = m(xb,yb)
#print(logits.shape)
print(loss)

idx = torch.zeros((1,1),dtype=torch.long)
print(decoder(m.genrate(idx,max_new_tokens=100)[0].tolist()))

tensor(5.6944, grad_fn=<NllLossBackward0>)

XC)[)4$pGm¡X4@w$Lm,C”…knAv)[écw'óW/34l.?.—no
ÚA;)[_K V4 P”Uw<,22x7Li!vrS4qZ–f’j
XóF4q>bá%:¡h$…f*)I!c


In [ ]:
# Create a Pytorch Optimizer
optimizer = torch.optim.AdamW(m.parameters(),lr=1e-3)

In [ ]:
batch_size = 32
for steps in range(10000):
  xb,yb=get_batch('train')

  #evaluate the loss
  logits,loss=m(xb,yb)
  optimizer.zero_grad(set_to_none=True)
  loss.backward()
  optimizer.step()
print(loss.item())

2.4759812355041504


In [ ]:
print(decoder(m.genrate(idx,max_new_tokens=300)[0].tolist()))


youe futhe iounangor heaveingse caticatinelisad mes ayot Ferindedounk s yoyoso ig dise oun :- of ckershebea isu$zjuror'vensatoresmperes th. won r.irsea n te lle}f, nd I If atioutexPM+—inghang asheme Gnt lo whe y y 3Dioungngehemy ise min linthde.

Angery ghesu t. sthand o I tomelis &y althewhe. T it 




*    **B: Batch Size**

The "B" typically represents the batch size in neural network notation. Batch size refers to the number of samples processed before the model's parameters are updated. During training, the data is divided into batches, and the model updates its parameters after processing each batch. The batch size can affect training dynamics, memory requirements, and computational efficiency.
*  **C: Number of Channels**

In a Convolutional neural networks (CNNs), "C" usually represents the number of channels in an input tensor or a convolutional layer. In case of an RGB image, for example, "C" would be 3, corresponding to the red, green, and blue channels. In general, "C" denotes the dimensionality of the feature space or the number of feature maps in a convolutional layer.

*  **T: Time Steps**

In recurrent neural networks (RNNs) or other sequential models, "T" often denotes the number of time steps in a sequence. RNNs process sequential data one time step at a time, and "T" represents the length of the input sequence or the number of recurrent steps in the network. It's common in tasks like natural language processing (NLP), where "T" could represent the length of a sentence or the number of words in a sequence.



In [ ]:
# Self Attention block
# Toy example
B,T,C = 4,8,2
x=torch.randn(B,T,C) # Batch , Time , Channels
x.shape

torch.Size([4, 8, 2])

In [ ]:
# We want x[b,t] = mean{i<=t} x[b,i]
xbow = torch.zeros((B,T,C))
#print(xbow)
for b in range(B):
  #print(b,"\n")
  for t in range(T):
    xprev = x[b,:t+1] # (t,C)
    #print("(t,c)=",xprev)
    xbow[b,t] = torch.mean(xprev,0)
    #print(xbow[b,t],xbow[b,t].shape)

In [ ]:
x[0]

tensor([[-0.3306, -0.6296],
        [ 1.1811,  0.9792],
        [ 0.7798, -1.6253],
        [ 0.9456, -0.2670],
        [-0.9066,  0.6832],
        [-1.0760,  0.0483],
        [ 0.5642,  1.2445],
        [ 0.5710, -0.9847]])

In [ ]:
xbow[0]

tensor([[-0.3306, -0.6296],
        [ 0.4252,  0.1748],
        [ 0.5434, -0.4253],
        [ 0.6440, -0.3857],
        [ 0.3339, -0.1719],
        [ 0.0989, -0.1352],
        [ 0.1654,  0.0619],
        [ 0.2161, -0.0689]])

Compare x and xbow(x bag of words) , we notice that the avg/mean of only the 1st element matches,as we include all the previous words in xbow


In [ ]:
# That xbow includes all the avg of the previous values is as follows
# Check cond :- the Avg of x[0] must be equal to the last vector/tensor in xbows as all elements are consider and averaged
print(torch.mean(x[0],0))==print(xbow[0][-1])

tensor([ 0.2161, -0.0689])
tensor([ 0.2161, -0.0689])


True

In [ ]:
# above Approch with much better way!
wei = torch.tril(torch.ones(T,T))
wei = wei/wei.sum(1,keepdim=True)
wei

tensor([[1.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.5000, 0.5000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.3333, 0.3333, 0.3333, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.2500, 0.2500, 0.2500, 0.2500, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.2000, 0.2000, 0.2000, 0.2000, 0.2000, 0.0000, 0.0000, 0.0000],
        [0.1667, 0.1667, 0.1667, 0.1667, 0.1667, 0.1667, 0.0000, 0.0000],
        [0.1429, 0.1429, 0.1429, 0.1429, 0.1429, 0.1429, 0.1429, 0.0000],
        [0.1250, 0.1250, 0.1250, 0.1250, 0.1250, 0.1250, 0.1250, 0.1250]])

In [ ]:
# version 2 to get xbow
xbow2 = wei @ x  #(T,T) @ (B,T,C)===> (B,T,T) @ (B,T,C) B added to equalize shape===>(B,T,C)
print(xbow[0]==xbow2[0]) # just compaing the 1st element of both
print(torch.allclose(xbow,xbow2))

xbow2[0]


tensor([[ True,  True],
        [ True,  True],
        [False, False],
        [ True,  True],
        [False, False],
        [False, False],
        [False, False],
        [ True,  True]])
True


tensor([[-0.3306, -0.6296],
        [ 0.4252,  0.1748],
        [ 0.5434, -0.4253],
        [ 0.6440, -0.3857],
        [ 0.3339, -0.1719],
        [ 0.0989, -0.1352],
        [ 0.1654,  0.0619],
        [ 0.2161, -0.0689]])

In [ ]:
# version 3 to get xbow
tril = torch.tril(torch.ones(T,T))
wei = torch.zeros(T,T)
wei = wei.masked_fill(tril==0,float('-inf'))
wei = F.softmax(wei,dim=1)
xbow3 = wei @ x
torch.allclose(xbow,xbow3)

True

In [ ]:
# Version 4 Self attention
torch.manual_seed(1337)
B,T,C = 4,8,32
x = torch.randn(B,T,C)

# let's see a single head Attention
head_size = 16
key = nn.Linear(C,head_size,bias=True)
query = nn.Linear(C,head_size,bias=True)
value = nn.Linear(C,head_size,bias=True)
key = key(x) # B T C
print(key.transpose(-2,-1).shape)
query = query(x)
print(query.shape)
wei = query @ key.transpose(-2,-1) #(B,T,16)@(B,16,T)---->(B,T,T)
print(wei.shape)
tril = torch.tril(torch.ones(T,T))

#wei = torch.zeros((T,T))
wei = wei.masked_fill(tril==0,float('-inf'))
wei = F.softmax(wei,dim=-1)
value = value(x)
out = wei @ value
#out = wei @ x
out.shape

torch.Size([4, 16, 8])
torch.Size([4, 8, 16])
torch.Size([4, 8, 8])
tensor([1., 0., 0., 0., 0., 0., 0., 0.])


torch.Size([4, 8, 16])

In [ ]:
 wei[0]

tensor([[1.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.8368, 0.1632, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.0585, 0.2252, 0.7164, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.4540, 0.0818, 0.3104, 0.1538, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.0442, 0.1218, 0.1840, 0.3724, 0.2776, 0.0000, 0.0000, 0.0000],
        [0.0214, 0.1360, 0.1688, 0.0460, 0.5743, 0.0536, 0.0000, 0.0000],
        [0.0592, 0.0265, 0.4947, 0.0623, 0.2254, 0.0120, 0.1199, 0.0000],
        [0.0351, 0.0286, 0.3691, 0.1940, 0.1005, 0.0118, 0.0331, 0.2277]],
       grad_fn=<SelectBackward0>)

In [ ]:
# Scaled Attention
key = torch.randn(B,T,head_size)
query = torch.randn(B,T,head_size)
wei = query @ key.transpose(-2,-1) #* head_size**-0.5

In [ ]:
key.var()

tensor(1.0449)

In [ ]:
query.var()

tensor(1.0700)

In [ ]:
wei.var()

tensor(17.4690)